If you decide to participate in the competition, save the **predictions.json and flops_and_params.json** files in a zip folder with your group name.

In [ ]:
# import packages
import pandas as pd
import numpy as np
import torch
import json

In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# unzip the files
# to find the file path, right click the filename and select copy path
# make sure don't change any file and folder name in the train_set and test_set
# !unzip /content/test_set.zip
# !unzip /content/train_set.zip

# Step 1

In [ ]:
import torch.nn as nn
# A simple CNN model
class SimpleCNN(nn.Module):
  def __init__(self, num_classes=5):
    super(SimpleCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
    self.fc1 = nn.Linear(2048, 512)
    self.fc2 = nn.Linear(512, 10)
    self.pool = nn.MaxPool2d(2, 2)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = self.pool(torch.relu(self.conv3(x)))
    x = x.view(-1, 2048)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = SimpleCNN()
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [ ]:
# NEW TOM + NATE BLOCK
import torch.nn as nn

class betterCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(betterCNN, self).__init__()
        
        # convolutional block
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        # convolutional block
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        
        # convolutional block
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(256)
        
        # pooling and dropout
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        
        # fully connected layers
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # 1 block
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool(x)
        
        # 2 block
        x = self.conv3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = torch.relu(x)
        x = self.pool(x)
        
        # 3 block
        x = self.conv5(x)
        x = self.bn5(x)
        x = torch.relu(x)
        x = self.conv6(x)
        x = self.bn6(x)
        x = torch.relu(x)
        x = self.pool(x)
        
        # flatten
        x = x.view(-1, 256 * 4 * 4)
        
        # fully connected layers
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

model = betterCNN(num_classes=5)
print(model)

# Step 2

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

# augmentation method can be added in training
# keep validation clean
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

val_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_dir = '/content/train_set'


train_dataset = datasets.ImageFolder(root=dataset_dir, transform=train_transform)
# you may desgin your validation set, and change the value of the "root" parameter.
val_dataset = datasets.ImageFolder(root=dataset_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000, shuffle=False)


In [ ]:
# NEW TOM + NATE BLOCK

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# training transform with augmentation
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# val transform (still without augmentation as in sample)
val_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset_dir = '/content/train_set'

full_dataset = datasets.ImageFolder(root=dataset_dir, transform=None)

# train/val split sizes
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

#print(f"Total images: {total_size}")
#print(f"Training images: {train_size}")
#print(f"Validation images: {val_size}")

# split into train/val indices
train_indices = list(range(0, train_size))
val_indices = list(range(train_size, total_size))

# train dataset w augmentation
train_dataset = datasets.ImageFolder(root=dataset_dir, transform=train_transform)
train_subset = torch.utils.data.Subset(train_dataset, train_indices)

# val dataset w/o augmentation
val_dataset = datasets.ImageFolder(root=dataset_dir, transform=val_transform)
val_subset = torch.utils.data.Subset(val_dataset, val_indices)

# data loaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

# Step 3

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# NEW TOM + NATE BLOCK

import torch.optim as optim
criterion = nn.CrossEntropyLoss(label_smoothing=0.1) # loss + add label smoothing
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001) # optimizer + add weight decay
# added learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',
    factor=0.5, 
    patience=3,
    verbose=True
)

# Step 4

In [ ]:
def train(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # you may consider to
        # save training log
        # save model checkpoints
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
train(model, train_loader, criterion, optimizer)

Epoch 1, Loss: 1.5363955263580595
Epoch 2, Loss: 1.4096388476235526
Epoch 3, Loss: 1.2820373880011695
Epoch 4, Loss: 1.150046474167279
Epoch 5, Loss: 1.1198387209858214


In [ ]:
# NEW TOM + NATE BLOCK

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=30):
    best_val_acc = 0.0
    patience_counter = 0
    early_stop_patience = 10
    
    train_losses = []
    val_accuracies = []
    
    for epoch in range(num_epochs):
        # train
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss = running_loss + loss.item()
            
            predictions = torch.max(outputs, 1)[1]
            train_total = train_total + labels.size(0)
            train_correct = train_correct + (predictions == labels).sum().item()
        
        avg_train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * train_correct / train_total
        train_losses.append(avg_train_loss)
        
        # val
        model.eval()
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                predictions = torch.max(outputs, 1)[1]
                val_total = val_total + labels.size(0)
                val_correct = val_correct + (predictions == labels).sum().item()
        
        val_acc = 100.0 * val_correct / val_total
        val_accuracies.append(val_acc)
        
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f" -> Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f" -> Val Acc: {val_acc:.2f}%")
        
        # new scheduler step
        scheduler.step(val_acc)
        
        # save state added
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"----> Best model saved ---> Val Acc: {val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter = patience_counter + 1
        
        # early stopping
        if patience_counter >= early_stop_patience:
            break
        
        print()
    
    return train_losses, val_accuracies

train_losses, val_accuracies = train(
    model, 
    train_loader, 
    val_loader, 
    criterion, 
    optimizer, 
    scheduler, 
    num_epochs=30
)


# Step 5

In [ ]:
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')
evaluate(model, val_loader)

Accuracy: 57.42686736722522%


In [ ]:
# NEW TOM + NATE BLOCK

def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            predictions = torch.max(outputs, 1)[1]
            total = total + labels.size(0)
            correct = correct + (predictions == labels).sum().item()
    
    accuracy = 100.0 * correct / total
    print(f'Val Accuracy: {accuracy:.2f}%')
    return accuracy

model.load_state_dict(torch.load('best_model.pth'))
evaluate(model, val_loader)

Please don't make any change after this line. The only parameters you may modify are those within the "test_transform" function.

In [ ]:
import os
from PIL import Image
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def filename2index(self, filename):
        return os.path.basename(filename).replace('.jpg', '')

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.filename2index(img_path)

In [ ]:
# if you save your model in advance, you can load it here.
# model.load_state_dict(torch.load('your_trained_model.pth'))

In [ ]:
test_folder = './test_set'
test_transform = transforms.Compose([
    transforms.Resize((32, 32)), # same image size applied for validation
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # same mean and std applied for validation
])
test_dataset = CustomImageDataset(test_folder, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

class_to_idx = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

In [ ]:
# Make predictions
def evaluate_model(model, test_loader, idx_to_class):
    all_predictions = {}
    with torch.no_grad():
        for inputs, index in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predicted_class = predicted.item()
            predicted_class_name = idx_to_class[predicted_class]
            all_predictions[index[0]] = predicted_class_name

    return all_predictions

predictions = evaluate_model(model, test_loader, idx_to_class)
with open('predictions.json', 'w') as json_file:
    json.dump(predictions, json_file, indent=4)

print("Evaluation completed and predictions saved.")

Evaluation completed and predictions saved.


In [ ]:
# you may need to install thop when you first run this code
# !pip install thop

In [ ]:
# Compute FLOPs using thop
import thop
input_tensor = test_dataset[0][0].unsqueeze(0) # must have exact same size of the data input (batch, channel, height, width)
flops, params = thop.profile(model, inputs=(input_tensor,))
print(f"FLOPs: {flops}")
print(f"Number of Parameters: {params}")
flops_and_params = {
    "FLOPs": flops,
    "Parameters": params
}

output_json_path = 'flops_and_params.json'

with open(output_json_path, 'w') as json_file:
    json.dump(flops_and_params, json_file, indent=4)

print(f"FLOPs and parameters have been saved to {output_json_path}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
FLOPs: 22751232.0
Number of Parameters: 1147466.0
FLOPs and parameters have been saved to flops_and_params.json
